# Neural Machine Traslation with Attention
Disclaimer: This notebook is an adopted version of [this repository](https://github.com/keon/seq2seq) and [this tutorial](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html).

In this tutorial we'll learn how to build recurrent neural network with attention mechanism to automatically translate from German to English! 

<img src="static/model.png" width=300 align="center"/>

Imports:

In [1]:
import os
import random
import numpy as np
import re

import torch
from torch import nn
from torch.nn import functional as F
from torch import optim

# for text processing
import spacy
import torchtext

Parameters:

In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

batch_size = 32
hidden_size = 512
embed_size = 256

## Data
1. To train NMT model in supervised manner we need dataset of parallel texts for 2 (or more) languages. Today we'll use **Multi30k** dataset from `torchtext` package. It's a small dataset containing exactly what we need - parallel sentences in German and English.

In [3]:
torchtext.datasets.Multi30k

torchtext.datasets.translation.Multi30k

2. To tokenize text we'll use `spacy`. Before using it you have to download German and English language packages:
```bash
python3 -m spacy download de
python3 -m spacy download en
```

3. To operate with text we'll use handy `torchtext` abstractions: [Field](https://torchtext.readthedocs.io/en/latest/data.html#torchtext.data.Field) and [BucketIterator](https://torchtext.readthedocs.io/en/latest/data.html#torchtext.data.BucketIterator). Read the docs on these classes for more details.

Let's combine all data loading and preparation stuff in one function:

In [4]:
def load_dataset(batch_size):
    spacy_de = spacy.load('de')
    spacy_en = spacy.load('en')
    url = re.compile('(<url>.*</url>)')

    def tokenize_de(text):
        return [tok.text for tok in spacy_de.tokenizer(url.sub('@URL@', text))]

    def tokenize_en(text):
        return [tok.text for tok in spacy_en.tokenizer(url.sub('@URL@', text))]

    DE = torchtext.data.Field(tokenize=tokenize_de, include_lengths=True, init_token='<sos>', eos_token='<eos>')
    EN = torchtext.data.Field(tokenize=tokenize_en, include_lengths=True, init_token='<sos>', eos_token='<eos>')
    
    train, val, test = torchtext.datasets.Multi30k.splits(exts=('.de', '.en'), fields=(DE, EN))
    DE.build_vocab(train.src, min_freq=2)
    EN.build_vocab(train.trg, max_size=10000)
    train_iter, val_iter, test_iter = torchtext.data.BucketIterator.splits(
            (train, val, test), batch_size=batch_size, repeat=False)
    return train_iter, val_iter, test_iter, DE, EN

In [5]:
train_iter, val_iter, test_iter, DE, EN = load_dataset(batch_size)
de_size, en_size = len(DE.vocab), len(EN.vocab)

Grab one batch:

In [6]:
batch = next(iter(train_iter))

In [14]:
batch.src[0].shape, batch.src[1].shape

(torch.Size([25, 32]), torch.Size([32]))

In [15]:
batch.trg[0].shape, batch.trg[1].shape

(torch.Size([22, 32]), torch.Size([32]))

In [22]:
batch.src[0].transpose(1, 0)[0], batch.src[1][0]

(tensor([   2,    5,  212,   45, 4426,    8,   16,   24,   74, 2629,   24, 1876,
            0,  110,    4,    3,    1,    1,    1,    1,    1,    1,    1,    1,
            1]), tensor(16))

In [23]:
batch.src[0]

tensor([[   2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
            2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
            2,    2,    2,    2,    2,    2,    2,    2],
        [   5,    5,  129,    5,   21,  126,    5,    5,    5,    5,    5,    5,
           21,    5,  587,    5,    5,    5,    5,   14,   21,    5,   76,   42,
            5,   14,   84,    5,   60,    5,    5,   21],
        [ 212,  402,   67, 1269, 1220,   29,   12,   85,   69,  134,  158, 4066,
           92,   12,   68,    0,   27,  706,   35,   17, 1949,   12,    7,  583,
           12,   17,    0,   12,   46,   12, 1903,   27],
        [  45, 1025,   59,   12,   85, 3667,    8,    7,   50,   12,   33,   27,
            7,   10,  219,   12, 2069,  166,   30,   10,  364,    7,    0,   19,
           11,    9,  995,   10,    7,  685,  158,   57],
        [4426,  185,   66,    8,   46,    0,   16,    6,   86,   30, 1129,    7,
          220,  880,   19,  310,    7, 

`batch` has 2 attributes: `src` and `trg`. Each contains tuple with numerical representation of the sentences with similar lengths (padded) and their original lengths:

In [7]:
print("input batch shape", batch.src[0].shape)
print("input batch lengths", batch.src[1])

input batch shape torch.Size([25, 32])
input batch lengths tensor([16, 20, 10, 14, 15, 22, 19, 18, 11, 11, 12, 16, 17, 14, 11, 15, 15, 10,
        14, 13, 12, 13, 10, 15, 12, 13, 22, 15, 25, 12, 11, 16])


Using `DE` and `EN` you can convert from string representation to numerical and back (`.stoi` and `.itos` methods):

In [8]:
print("encoded sentence", batch.src[0][:, 0])
print("encoded sentence", [DE.vocab.itos[token] for token in batch.src[0][:, 0]])

encoded sentence tensor([   2,    5,  212,   45, 4426,    8,   16,   24,   74, 2629,   24, 1876,
           0,  110,    4,    3,    1,    1,    1,    1,    1,    1,    1,    1,
           1])
encoded sentence ['<sos>', 'Ein', 'Bild', 'eines', 'Bürgersteigs', ',', 'der', 'von', 'zwei', 'Reihen', 'von', 'Säulen', '<unk>', 'wird', '.', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']


Now have data, it's time build models!

# Task 1 (2 points). Seq2Seq
A **Sequence to Sequence** network, or **seq2seq** network, or **Encoder-Decoder** network, is a model consisting of two RNNs called the encoder and decoder. The encoder reads an input sequence and outputs a single vector, and the decoder reads that vector to produce an output sequence. Unlike sequence prediction with a single RNN, where every input corresponds to an output, the seq2seq model frees us from sequence length and order, which makes it ideal for translation between two languages.

With a seq2seq model the encoder creates a single vector which, in the ideal case, encodes the “meaning” of the input sequence into a single vector — a single point in some N dimensional space of sentences.

<img src="static/seq2seq.png" width=1000 align="center"/>

### Encoder
The encoder of a seq2seq network is a bidirectional GRU RNN that outputs some value for every word from the input sentence. For every input word the encoder outputs a vector and a hidden state, and uses the hidden state for the next input word. To keep hidden size of fixed shape we sum outputs over 2 directions.

**Note:** here we use [nn.GRU](https://pytorch.org/docs/stable/nn.html#torch.nn.GRU), not [nn.GRUCell](https://pytorch.org/docs/stable/nn.html#torch.nn.GRUCell). Read the docs to understand the differences.

In [13]:
class Encoder(nn.Module):
    def __init__(self, input_size, embed_size, hidden_size,
                 n_layers=1, dropout=0.5):
        super().__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.embed_size = embed_size
        self.embed = nn.Embedding(input_size, embed_size)
        self.gru = nn.GRU(embed_size, hidden_size, n_layers, dropout=dropout, bidirectional=True)

    def forward(self, src, hidden=None):
        """Encodes input sequence

        Args:
            src (torch tensor of shape (t, b)): input sequence
            hidden (torch tensor of shape (n_layers * n_directions, b, h)): prev hidden state (can be None)
        
        Returns:
            outputs (torch tensor of shape (t, b, h)): encoded sequence (dicrections are summed)
            hidden (torch tensor of shape (n_layers * n_directions, b, h)): hidden state
        """
        # embedded = (t, b, embed_size)
        embedded = self.embed(src)## embed input
        
        # output = (t, b, n_directions * h)
        # hidden = (n_layers * n_directions, b, h)
        outputs, hidden = self.gru(embedded, hidden)## forward recurrent unit
        
        # sum bidirectional outputs
        # output = (t, b, n_directions, h)
        outputs = outputs.view(outputs.shape[0], outputs.shape[1], 2, self.hidden_size)
        # output = (t, b, h)
        outputs = outputs.sum(dim=2)
        
        return outputs, hidden

### Decoder
In the simplest seq2seq decoder uses only last output of the encoder. This last output is sometimes called the context vector as it encodes context from the entire sequence. This context vector is used as the initial hidden state of the decoder. At every step of decoding, the decoder is given an input token and hidden state. The initial input token is the start-of-string <SOS> token, and the first hidden state is the context vector (the encoder’s last hidden state).
    
But in this tutorial we'll pump our decoder with attention mechanism!

### Decoder with attention

If only the context vector is passed betweeen the encoder and decoder, that single vector carries the burden of encoding the entire sentence.

Attention allows the decoder network to “focus” on a different part of the encoder’s outputs for every step of the decoder’s own outputs. First we calculate a set of attention weights. These will be multiplied by the encoder output vectors to create a weighted combination. The result should contain information about that specific part of the input sequence, and thus help the decoder choose the right output words.

<img src="static/attention.png" width=500 align="center"/>

Calculating the attention weights is done with another feed-forward layer, using the decoder’s input and hidden state as inputs. Below you can find short description of the so-called Bahdanau attention mechanism (details can be found in the paper [Neural Machine Translation by Jointly Learning to Align and Translate](https://arxiv.org/abs/1409.0473)).

Attention weights ($\alpha_{ij}$):

$$
    e_{ij} = f(s_{i-1}, h_j) = v \tanh(W [s_{i-1}, h_j] + b) \\
    \alpha_{ij} = softmax(e_{ij}) = \frac{\exp{e_{ij}}}{\sum_j{\exp{e_{ij}}}}
$$

Here $s_{i-1}$ - hidden state of decoder (`hidden`), $h_j$ - hidden state of encoder (`encoder_outputs`), $v$, $W$, $b$ - learnable parameters.

Let's implement it:

In [29]:
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
        
        # setup attention parameters
        self.v = nn.Parameter(torch.zeros(hidden_size))
#         self.v = nn.Parameter(torch.zeros(hidden_size * 2))
        
        stdv = 1. / np.sqrt(self.v.shape[0])
        self.v.data.uniform_(-stdv, stdv)

    def forward(self, hidden, encoder_outputs):
        """Calculates attention weights

        Args:
            hidden (torch tensor of shape (b, h)): prev hidden state (can be None)
            encoder_outputs (torch tensor of shape (t, b, h)): encoded sequence
        
        Returns:
            attn_weights (torch tensor of shape (b, 1, t)): attention weights
        """ 
        
        timestep = encoder_outputs.shape[0]
        h = hidden.repeat(timestep, 1, 1).transpose(0, 1)  # [B*T*H]
        encoder_outputs = encoder_outputs.transpose(0, 1)  # [B*T*H]
        
        # [B*T*2H]->[B*T*H]
        ## concat h and encoder_outputs, feed to self.attn and then to softmax 
        energy = nn.Softmax(dim=1)(self.attn(torch.cat([h, encoder_outputs], dim=2))) 
#         energy = self.attn(torch.cat([h, encoder_outputs], dim=2))
        energy = energy.transpose(1, 2)  # [B*H*T]
        
        v = self.v.repeat(encoder_outputs.shape[0], 1).unsqueeze(1)  # [B*1*H]
        attn_weights = torch.bmm(v, energy) ## multiply by v vector to get shape [B*1*T]
#         attn_weights = nn.Softmax(torch.bmm(v, energy), dim=1) ## multiply by v vector to get shape [B*1*T]
        
        return attn_weights
    
# output = (t, b, h)
# hidden = (., b, h)

Now let's insert attention mechanism to decoder:

In [15]:
class Decoder(nn.Module):
    def __init__(self, embed_size, hidden_size, output_size,
                 n_layers=1, dropout=0.2):
        super().__init__()
        
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers

        self.embed = nn.Embedding(output_size, embed_size)
        self.dropout = nn.Dropout(dropout, inplace=True)
        self.attention = Attention(hidden_size)
        self.gru = nn.GRU(hidden_size + embed_size, hidden_size, n_layers, dropout=dropout)
        self.out = nn.Linear(hidden_size * 2, output_size)

    def forward(self, input, last_hidden, encoder_outputs):
        """Decodes with attention token by token

        Args:
            input (torch tensor of shape (b,)): input token
            last_hidden (torch tensor of shape (1, b, h)): last hidden
            encoder_outputs (torch tensor of shape (t, b, h)): encoded sequence
        
        Returns:
            output (torch tensor of shape (b, vocab_size)): ouput token distribution
            hidden (torch tensor of shape (1, b, h)): hidden state
            attn_weights (torch tensor of shape (b, 1, t)): attention weights
        """
        # get the embedding of the current input word (last output word)
        embedded = self.embed(input).unsqueeze(0)  # (1,B,N)
        embedded = self.dropout(embedded)
        
        # calculate attention weights and apply to encoder outputs
        attn_weights = self.attention(last_hidden[-1], encoder_outputs)  # (B,1,T)
        context = torch.bmm(attn_weights, encoder_outputs.transpose(1, 0))## apply attention weights to encoder_outputs to get shape # (B,1,N) (don't forget to transpose encoder_outputs)
        context = context.transpose(0, 1)  # (1,B,N)
        
        # combine embedded input word and attended context, run through RNN
        rnn_input = torch.cat([embedded, context], 2)
        output, hidden = self.gru(rnn_input, last_hidden)## forward recurrent unit 
        output = output.squeeze(0)  # (1,B,N) -> (B,N)
        
        context = context.squeeze(0)
        output = self.out(torch.cat([output, context], 1)) # (B, output_size)
        output = F.log_softmax(output, dim=1) # (B, output_size)

        return output, hidden, attn_weights

## Wrap in a single Seq2Seq model

In [16]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, sos_token, max_len):
        """Sequence-to-sequence inference

        Args:
            src (torch tensor of shape (t, b)): input sequence
        
        Returns:
            outputs (torch tensor of shape (b, vocab_size)): ouput token distribution
        """
        device = src.device
        
        batch_size = src.shape[1]
        vocab_size = self.decoder.output_size
        outputs = torch.zeros(max_len, batch_size, vocab_size).to(device)

        # output = (t, b, h)
        # hidden = (., b, h)
        encoder_output, hidden = self.encoder(src)
        hidden = hidden[:self.decoder.n_layers]
        
        output = torch.full((batch_size,), sos_token, dtype=torch.long).to(device)
        
        for t in range(1, max_len):
            # output = (b, output_size)
            # hidden = (1, b, h)
            # attn_weights = (b, 1, t)
            output, hidden, attn_weights = self.decoder(output, hidden, encoder_output) ## apply decoder
            outputs[t] = output
            
            top1 = output.data.max(1)[1]
            output = top1.to(device)
            # output = (b, )
            
        return outputs

In [27]:
0.05 ** 40

9.094947017729302e-53

In [30]:
np.log(0.005) 

-5.298317366548036

In [24]:
o = torch.rand((2, 5))

In [25]:
o

tensor([[0.1700, 0.2133, 0.8066, 0.3942, 0.5051],
        [0.4614, 0.7593, 0.0098, 0.0849, 0.7965]])

In [31]:
o.data.topk(3,1)

(tensor([[0.8066, 0.5051, 0.3942],
         [0.7965, 0.7593, 0.4614]]), tensor([[2, 4, 3],
         [4, 1, 0]]))

In [32]:
o.clone()

tensor([[0.1700, 0.2133, 0.8066, 0.3942, 0.5051],
        [0.4614, 0.7593, 0.0098, 0.0849, 0.7965]])

## Task 2 (1 point). Train-loop

Parameters:

In [17]:
epochs = 100
lr = 0.0001
grad_clip = 10.0

Model:

In [30]:
encoder = Encoder(de_size, embed_size, hidden_size, n_layers=2, dropout=0.5)
decoder = Decoder(embed_size, hidden_size, en_size, n_layers=1, dropout=0.0)
seq2seq = Seq2Seq(encoder, decoder).to(device)

optimizer = optim.Adam(seq2seq.parameters(), lr=lr)
print(seq2seq)

trg_sos_token = EN.vocab.stoi['<sos>']

Seq2Seq(
  (encoder): Encoder(
    (embed): Embedding(8014, 256)
    (gru): GRU(256, 512, num_layers=2, dropout=0.5, bidirectional=True)
  )
  (decoder): Decoder(
    (embed): Embedding(10004, 256)
    (dropout): Dropout(p=0.0, inplace)
    (attention): Attention(
      (attn): Linear(in_features=1024, out_features=512, bias=True)
    )
    (gru): GRU(768, 512)
    (out): Linear(in_features=1024, out_features=10004, bias=True)
  )
)


Wrapped `train` and `evaluate` ops:

In [31]:
def evaluate(model, val_iter, vocab_size, device, DE, EN):
    model.eval()
    pad = EN.vocab.stoi['<pad>']
    total_loss = 0
    for b, batch in enumerate(val_iter):
        src, len_src = batch.src
        trg, len_trg = batch.trg

        src, trg = src.to(device), trg.to(device)
        
        with torch.no_grad():
            # apply model
            output = model(src, trg_sos_token, trg.shape[0]) ## your code here
            
            # calculate nll loss
            # 1) don't take into account first token (it's always <sos>)
            # 2) don't take into account pad token (ignore_index argument)
            loss = F.nll_loss(output[1:].view(-1, vocab_size), trg[1:].view(-1), ignore_index=pad)## your code here
            
            total_loss += loss.item()
    return total_loss / len(val_iter)


def train(e, model, optimizer, train_iter, vocab_size, device, grad_clip, DE, EN):
    model.train()
    total_loss = 0
    pad = EN.vocab.stoi['<pad>']
    for b, batch in enumerate(train_iter):
        src, len_src = batch.src
        trg, len_trg = batch.trg
        src, trg = src.to(device), trg.to(device)
        optimizer.zero_grad()
        
        # apply model
        output = model(src, trg_sos_token, trg.shape[0]) ## your code here
        
        # calculate nll loss
        # 1) don't take into account first token (it's always <sos>)
        # 2) don't take into account pad token (ignore_index argument)
        loss = F.nll_loss(output[1:].view(-1, vocab_size), trg[1:].view(-1), ignore_index=pad)## your code here
        
        loss.backward()
        
        # clip gradients using nn.utils.clip_grad_norm_ by `grad_clip` value
        nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
        
        optimizer.step()
        total_loss += loss.item()

        if b % 10 == 0 and b != 0:
            total_loss = total_loss / 10
            print("[%d][loss:%5.2f][pp:%5.2f]" % (b, total_loss, np.exp(total_loss)))
            total_loss = 0

To monitor how our training is going on, let's translate fixed batch of sentences every epoch:

In [32]:
fixed_test_batch = next(iter(test_iter))

In [33]:
def show_translations(seq2seq, batch, device, trg_sos_token, max_len=10, max_examples=5):
    sentence_encoded = batch.src[0].to(device)
    
    for example_i in range(min(batch.src[0].shape[1], max_examples)):
        input_encoded = batch.src[0][:, example_i]
        input = " ".join([DE.vocab.itos[index] for index in input_encoded][1:batch.src[1][example_i]])
        
        result_encoded = seq2seq(sentence_encoded, trg_sos_token, max_len)
        result_encoded = result_encoded.argmax(dim=2)[:, example_i]
        pred = " ".join([EN.vocab.itos[index] for index in result_encoded][1:])

        gt_encoded = batch.trg[0][:, example_i]
        gt = " ".join([EN.vocab.itos[index] for index in gt_encoded][1:batch.trg[1][example_i]])
        
        print("input:\t", input)
        print("pred:\t", pred)
        print("gt:\t", gt)
        print()

Run train-loop:

In [ ]:
best_val_loss = None

for e in range(1, epochs + 1):
    train(e, seq2seq, optimizer, train_iter, en_size, device, grad_clip, DE, EN)
    val_loss = evaluate(seq2seq, val_iter, en_size, device, DE, EN)
    print("[Epoch:%d] val_loss:%5.3f | val_pp:%5.2fS" % (e, val_loss, np.exp(val_loss)))

    # save the model if the validation loss is the best we've seen so far.
    if not best_val_loss or val_loss < best_val_loss:
        print("[!] saving model...")
        if not os.path.isdir("weights"):
            os.makedirs("weights")
        torch.save(seq2seq.state_dict(), 'weights/seq2seq_%d.pth' % (e))
        best_val_loss = val_loss
    
    print("Samples from test:")
    show_translations(seq2seq, fixed_test_batch, device, trg_sos_token, max_len=20, max_examples=5)
    print()
        
test_loss = evaluate(seq2seq, test_iter, en_size, device, DE, EN)
print("[TEST] loss:%5.2f" % test_loss)

[10][loss: 9.45][pp:12711.48]
[20][loss: 6.78][pp:882.92]
[30][loss: 6.03][pp:415.79]
[40][loss: 5.73][pp:308.83]
[50][loss: 5.56][pp:258.87]
[60][loss: 5.43][pp:227.08]
[70][loss: 5.41][pp:223.26]
[80][loss: 5.43][pp:228.80]
[90][loss: 5.30][pp:201.19]
[100][loss: 5.33][pp:205.74]
[110][loss: 5.28][pp:197.16]
[120][loss: 5.24][pp:188.01]
[130][loss: 5.30][pp:199.88]
[140][loss: 5.12][pp:166.87]
[150][loss: 5.13][pp:168.74]
[160][loss: 5.18][pp:178.02]
[170][loss: 5.12][pp:167.22]
[180][loss: 5.17][pp:175.06]
[190][loss: 5.03][pp:152.89]
[200][loss: 5.10][pp:164.33]
[210][loss: 5.09][pp:163.19]
[220][loss: 5.07][pp:159.64]
[230][loss: 5.06][pp:157.89]
[240][loss: 4.97][pp:143.88]
[250][loss: 5.02][pp:152.04]
[260][loss: 5.07][pp:158.58]
[270][loss: 5.06][pp:157.90]
[280][loss: 4.97][pp:144.18]
[290][loss: 4.88][pp:131.07]
[300][loss: 4.96][pp:142.53]
[310][loss: 4.98][pp:145.88]
[320][loss: 4.92][pp:136.75]
[330][loss: 4.86][pp:129.22]
[340][loss: 4.84][pp:126.51]
[350][loss: 4.89][pp:

[310][loss: 3.49][pp:32.69]
[320][loss: 3.62][pp:37.51]
[330][loss: 3.48][pp:32.37]
[340][loss: 3.57][pp:35.61]
[350][loss: 3.48][pp:32.52]
[360][loss: 3.67][pp:39.08]
[370][loss: 3.64][pp:37.94]
[380][loss: 3.53][pp:34.01]
[390][loss: 3.54][pp:34.33]
[400][loss: 3.51][pp:33.51]
[410][loss: 3.55][pp:34.89]
[420][loss: 3.46][pp:31.81]
[430][loss: 3.52][pp:33.66]
[440][loss: 3.50][pp:33.09]
[450][loss: 3.58][pp:35.93]
[460][loss: 3.39][pp:29.55]
[470][loss: 3.48][pp:32.35]
[480][loss: 3.45][pp:31.40]
[490][loss: 3.57][pp:35.59]
[500][loss: 3.64][pp:38.03]
[510][loss: 3.52][pp:33.66]
[520][loss: 3.54][pp:34.61]
[530][loss: 3.42][pp:30.62]
[540][loss: 3.49][pp:32.72]
[550][loss: 3.45][pp:31.40]
[560][loss: 3.50][pp:33.21]
[570][loss: 3.39][pp:29.70]
[580][loss: 3.44][pp:31.30]
[590][loss: 3.49][pp:32.65]
[600][loss: 3.41][pp:30.18]
[610][loss: 3.56][pp:35.29]
[620][loss: 3.59][pp:36.12]
[630][loss: 3.52][pp:33.92]
[640][loss: 3.58][pp:35.72]
[650][loss: 3.62][pp:37.20]
[660][loss: 3.44][pp

[620][loss: 2.94][pp:18.87]
[630][loss: 3.04][pp:20.87]
[640][loss: 2.90][pp:18.19]
[650][loss: 2.97][pp:19.53]
[660][loss: 3.02][pp:20.55]
[670][loss: 2.99][pp:19.85]
[680][loss: 3.00][pp:20.04]
[690][loss: 2.99][pp:19.97]
[700][loss: 3.04][pp:20.97]
[710][loss: 3.00][pp:20.19]
[720][loss: 2.95][pp:19.02]
[730][loss: 2.99][pp:19.82]
[740][loss: 3.03][pp:20.69]
[750][loss: 2.96][pp:19.38]
[760][loss: 2.99][pp:19.88]
[770][loss: 3.07][pp:21.59]
[780][loss: 3.11][pp:22.32]
[790][loss: 2.97][pp:19.40]
[800][loss: 2.92][pp:18.55]
[810][loss: 3.02][pp:20.54]
[820][loss: 3.02][pp:20.46]
[830][loss: 2.97][pp:19.49]
[840][loss: 2.91][pp:18.43]
[850][loss: 2.85][pp:17.25]
[860][loss: 2.99][pp:19.79]
[870][loss: 2.94][pp:18.87]
[880][loss: 3.00][pp:20.07]
[890][loss: 2.92][pp:18.52]
[900][loss: 2.96][pp:19.24]
[Epoch:5] val_loss:3.365 | val_pp:28.95S
[!] saving model...
Samples from test:
input:	 Arbeiter diskutieren neben den Schienen . <eos>
pred:	 Workers workers to to next to the . <eos> <eo

input:	 Arbeiter diskutieren neben den Schienen . <eos>
pred:	 Workers workers the to next to the . <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos>
gt:	 Construction workers having a discussion by the tracks . <eos>

input:	 Zwei Jungen spielen gegeneinander Fußball . <eos>
pred:	 Two boys are playing soccer soccer soccer . <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos>
gt:	 Two boys play soccer against each other . <eos>

input:	 Zwei Fußballmannschaften auf dem Feld . <eos>
pred:	 Two players players on the the . <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos>
gt:	 Two soccer teams are on the field . <eos>

input:	 Ein hellbrauner Hund läuft bergauf . <eos>
pred:	 A tan tan runs tan tan . <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos>
gt:	 A light brown dog is running up . <eos>

input:	 Leute bewundern ein Kunstwerk . <eos>
pred:	 People art a a art . <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos>

[10][loss: 2.55][pp:12.78]
[20][loss: 2.37][pp:10.65]
[30][loss: 2.28][pp: 9.77]
[40][loss: 2.26][pp: 9.61]
[50][loss: 2.30][pp: 9.99]
[60][loss: 2.25][pp: 9.53]
[70][loss: 2.34][pp:10.38]
[80][loss: 2.27][pp: 9.71]
[90][loss: 2.28][pp: 9.78]
[100][loss: 2.34][pp:10.33]
[110][loss: 2.32][pp:10.20]
[120][loss: 2.27][pp: 9.73]
[130][loss: 2.23][pp: 9.34]
[140][loss: 2.28][pp: 9.79]
[150][loss: 2.42][pp:11.27]
[160][loss: 2.34][pp:10.40]
[170][loss: 2.27][pp: 9.63]
[180][loss: 2.32][pp:10.21]
[190][loss: 2.27][pp: 9.65]
[200][loss: 2.33][pp:10.27]
[210][loss: 2.25][pp: 9.46]
[220][loss: 2.33][pp:10.29]
[230][loss: 2.39][pp:10.88]
[240][loss: 2.36][pp:10.58]
[250][loss: 2.36][pp:10.56]
[260][loss: 2.40][pp:11.07]
[270][loss: 2.31][pp:10.03]
[280][loss: 2.34][pp:10.43]
[290][loss: 2.35][pp:10.46]
[300][loss: 2.34][pp:10.43]
[310][loss: 2.29][pp: 9.92]
[320][loss: 2.37][pp:10.73]
[330][loss: 2.26][pp: 9.57]
[340][loss: 2.40][pp:11.04]
[350][loss: 2.25][pp: 9.53]
[360][loss: 2.33][pp:10.24]
[

[340][loss: 2.19][pp: 8.92]
[350][loss: 2.19][pp: 8.92]
[360][loss: 2.16][pp: 8.67]
[370][loss: 2.21][pp: 9.10]
[380][loss: 2.17][pp: 8.77]
[390][loss: 2.21][pp: 9.07]
[400][loss: 2.27][pp: 9.64]
[410][loss: 2.17][pp: 8.72]
[420][loss: 2.19][pp: 8.90]
[430][loss: 2.21][pp: 9.08]
[440][loss: 2.21][pp: 9.11]
[450][loss: 2.23][pp: 9.30]
[460][loss: 2.10][pp: 8.18]
[470][loss: 2.30][pp:10.00]
[480][loss: 2.22][pp: 9.19]
[490][loss: 2.26][pp: 9.57]
[500][loss: 2.20][pp: 8.98]
[510][loss: 2.24][pp: 9.42]
[520][loss: 2.20][pp: 8.99]
[530][loss: 2.20][pp: 9.04]
[540][loss: 2.19][pp: 8.92]
[550][loss: 2.26][pp: 9.55]
[560][loss: 2.18][pp: 8.82]
[570][loss: 2.26][pp: 9.56]
[580][loss: 2.31][pp:10.04]
[590][loss: 2.19][pp: 8.90]
[600][loss: 2.26][pp: 9.60]
[610][loss: 2.20][pp: 9.07]
[620][loss: 2.17][pp: 8.79]
[630][loss: 2.20][pp: 9.00]
[640][loss: 2.17][pp: 8.77]
[650][loss: 2.22][pp: 9.18]
[660][loss: 2.25][pp: 9.49]
[670][loss: 2.26][pp: 9.60]
[680][loss: 2.19][pp: 8.91]
[690][loss: 2.23][pp

[670][loss: 2.12][pp: 8.35]
[680][loss: 2.05][pp: 7.79]
[690][loss: 2.10][pp: 8.19]
[700][loss: 2.01][pp: 7.50]
[710][loss: 2.14][pp: 8.47]
[720][loss: 2.08][pp: 8.02]
[730][loss: 2.09][pp: 8.06]
[740][loss: 2.09][pp: 8.11]
[750][loss: 2.11][pp: 8.21]
[760][loss: 2.11][pp: 8.23]
[770][loss: 2.06][pp: 7.83]
[780][loss: 2.07][pp: 7.90]
[790][loss: 2.11][pp: 8.28]
[800][loss: 2.07][pp: 7.89]
[810][loss: 2.13][pp: 8.40]
[820][loss: 1.98][pp: 7.21]
[830][loss: 2.05][pp: 7.78]
[840][loss: 2.14][pp: 8.46]
[850][loss: 2.12][pp: 8.34]
[860][loss: 2.13][pp: 8.40]
[870][loss: 2.13][pp: 8.42]
[880][loss: 2.15][pp: 8.62]
[890][loss: 2.08][pp: 7.97]
[900][loss: 2.06][pp: 7.83]
[Epoch:14] val_loss:3.403 | val_pp:30.04S
Samples from test:
input:	 Arbeiter diskutieren neben den Schienen . <eos>
pred:	 Workers are their to next to their truck . . <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos>
gt:	 Construction workers having a discussion by the tracks . <eos>

input:	 Zwei Jungen spielen gegenein

input:	 Zwei Jungen spielen gegeneinander Fußball . <eos>
pred:	 Two boys are playing playing soccer . <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos>
gt:	 Two boys play soccer against each other . <eos>

input:	 Zwei Fußballmannschaften auf dem Feld . <eos>
pred:	 Two two players are on the the . <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos>
gt:	 Two soccer teams are on the field . <eos>

input:	 Ein hellbrauner Hund läuft bergauf . <eos>
pred:	 A tan tan runs running tan . <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos>
gt:	 A light brown dog is running up . <eos>

input:	 Leute bewundern ein Kunstwerk . <eos>
pred:	 People are a art art art . <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos>
gt:	 People are admiring a work of art . <eos>


[10][loss: 2.03][pp: 7.62]
[20][loss: 1.83][pp: 6.24]
[30][loss: 1.93][pp: 6.87]
[40][loss: 1.89][pp: 6.65]
[50][loss: 1.88][pp: 6.53]
[60][loss: 1.82][pp: 6

[40][loss: 1.77][pp: 5.88]
[50][loss: 1.73][pp: 5.66]
[60][loss: 1.80][pp: 6.02]
[70][loss: 1.82][pp: 6.16]
[80][loss: 1.69][pp: 5.43]
[90][loss: 1.80][pp: 6.06]
[100][loss: 1.84][pp: 6.28]
[110][loss: 1.75][pp: 5.74]
[120][loss: 1.81][pp: 6.10]
[130][loss: 1.72][pp: 5.61]
[140][loss: 1.83][pp: 6.24]
[150][loss: 1.74][pp: 5.67]
[160][loss: 1.78][pp: 5.96]
[170][loss: 1.71][pp: 5.51]
[180][loss: 1.83][pp: 6.22]
[190][loss: 1.80][pp: 6.05]
[200][loss: 1.83][pp: 6.26]
[210][loss: 1.80][pp: 6.05]
[220][loss: 1.80][pp: 6.05]
[230][loss: 1.76][pp: 5.83]
[240][loss: 1.75][pp: 5.77]
[250][loss: 1.76][pp: 5.81]
[260][loss: 1.73][pp: 5.62]
[270][loss: 1.76][pp: 5.82]
[280][loss: 1.83][pp: 6.23]
[290][loss: 1.81][pp: 6.09]
[300][loss: 1.80][pp: 6.03]
[310][loss: 1.75][pp: 5.77]
[320][loss: 1.84][pp: 6.32]
[330][loss: 1.83][pp: 6.26]
[340][loss: 1.76][pp: 5.84]
[350][loss: 1.85][pp: 6.33]
[360][loss: 1.83][pp: 6.25]
[370][loss: 1.84][pp: 6.29]
[380][loss: 1.81][pp: 6.13]
[390][loss: 1.86][pp: 6.46

Yeah, we did it!